In [1]:
import os
import skimage.io as io
import pandas as pd
import numpy as np
from collections import OrderedDict

## Import TensorFlow and enable eager execution

In [2]:
import tensorflow as tf
tf.enable_eager_execution()

## Settings

In [3]:
COCO_ROOT = '/data/COCO/'
CKPT_DIR = 'logs/person_coco_aspp/ckpt'
THRESHOLD = 0.9

## Import COCO API

In [4]:
import sys
sys.path.insert(0, os.path.join(COCO_ROOT, 'cocoapi/PythonAPI'))
from pycocotools.coco import COCO

## Load COCO image list

In [5]:
ann_file_fpath = os.path.join(COCO_ROOT, 'annotations', 'instances_val2017.json')
coco = COCO(ann_file_fpath)
cat_ids = coco.getCatIds(catNms=['person'])
img_list = coco.getImgIds(catIds=cat_ids)
print('Number of images: {}'.format(len(img_list)))

loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
Number of images: 2693


## Function to encode segmentation map using RLE

In [6]:
def mask_to_rle(mask):
    mask_flat = mask.flatten('F')
    flag = 0
    rle_list = list()
    for i in range(mask_flat.shape[0]):
        if flag == 0:
            if mask_flat[i] == 1:
                flag = 1
                starts = i+1
                rle_list.append(starts)
        else:
            if mask_flat[i] == 0:
                flag = 0
                ends = i
                rle_list.append(ends-starts+1)
    if flag == 1:
        ends = mask_flat.shape[0]
        rle_list.append(ends-starts+1)
    #sanity check
    if len(rle_list) % 2 != 0:
        print('NG')
    if len(rle_list) == 0:
        rle = np.nan
    else:
        rle = ' '.join(map(str,rle_list))
    return rle

## Load model and restore from a checkpoint

In [7]:
import model
net = model.Model()

ckpt = tf.train.Checkpoint(model=net)
ckpt.restore(tf.train.latest_checkpoint(CKPT_DIR))

## Single image inference function

In [8]:
def inference(img):
    if len(img.shape) == 2:
        img = np.tile(img[..., None], (1, 1, 3))
    img = img[None, ...].astype(np.float32) / np.float32(255.)
    
    img640 = np.zeros([1, 640, 640, 3], np.float32)
    img640[:, :img.shape[1], :img.shape[2], :] = img

    logits = net(img640, is_training=False)
    img_out = tf.sigmoid(logits).numpy()
    
    mask_out = (img_out[0, :, :, 0] > THRESHOLD).astype(np.uint8)
    mask_out = mask_out[:img.shape[1], :img.shape[2]]
    
    return mask_out

## Run inference on all test images and encode the result

In [9]:
submit_name_arr = list()
submit_rle_arr = list()

for counter, img_id in enumerate(img_list):
    img_data = coco.loadImgs(img_id)[0]
    img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])
    
    if counter % 10 == 0:
        print('{} : {}'.format(counter, img_fname))
        
    img = io.imread(os.path.join(COCO_ROOT, img_fname))
    mask_out = inference(img)

    rle = mask_to_rle(mask_out)

    submit_name_arr.append(img_fname)
    submit_rle_arr.append(rle)        

0 : val2017/000000532481.jpg
Instructions for updating:
Colocations handled automatically by placer.
10 : val2017/000000401446.jpg
20 : val2017/000000475191.jpg
30 : val2017/000000122962.jpg
40 : val2017/000000442480.jpg
...


## Create Pandas DataFrame and save as CSV

In [10]:
submit_df = pd.DataFrame(OrderedDict({
    'ImageId': submit_name_arr,
    'EncodedPixels': submit_rle_arr,
}))

submit_df.to_csv('submit.csv', index=False)

submit_df.head()

,ImageId,EncodedPixels
0,val2017/000000532481.jpg,116073 1 116498 2 116923 4 117348 6 117774 7 1...
1,val2017/000000458755.jpg,14009 1 14487 4 14966 5 14988 2 15445 6 15466 ...
2,val2017/000000385029.jpg,2329 10 2809 11 3289 12 3769 13 4249 13 4728 1...
3,val2017/000000311303.jpg,11938 1 12365 1 12792 1 13219 1 13646 1 14073 ...
4,val2017/000000393226.jpg,46405 3 46881 10 47358 14 47836 17 48313 21 48...
